# TensorFlow Notebook (Tensorboard)

> Tensorflowの自主学習メモです。組織・団体を代表するものではありません。

Tensorboardのより詳細な使い方は、[TensorBoard README](https://github.com/tensorflow/tensorflow/blob/r0.11/tensorflow/tensorboard/README.md)。

### データの読み込み

* summaryデータを読み込み可視化する
* [tf.merge_all_summaries](https://www.tensorflow.org/versions/r0.11/api_docs/python/train.html#merge_all_summaries)を利用
* [tf.train.SummaryWriter](https://www.tensorflow.org/versions/r0.11/api_docs/python/train.html#SummaryWriter)を利用

### 実行例

* [Simple MNIST tutorial](http://tensorflow.org/tutorials/mnist/beginners/index.md)をsummary情報が出るように改造

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

fake_data = False
max_steps = 1000
learning_rate = 0.001
dropout = 0.9
data_dir = '/tmp/data'
summaries_dir = '/tmp/mnist_logs'

In [2]:
def train():
  # Import data
  mnist = input_data.read_data_sets(data_dir,
                                    one_hot=True,
                                    fake_data=fake_data)

  sess = tf.InteractiveSession()

  # Create a multilayer model.

  # Input placeholders
  with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')

  with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.image_summary('input', image_shaped_input, 10)

  # We can't initialize these variables to 0 - the network will get stuck.
  def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

  def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

  def variable_summaries(var, name):
    """Attach a lot of summaries to a Tensor."""
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.scalar_summary('mean/' + name, mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.scalar_summary('stddev/' + name, stddev)
      tf.scalar_summary('max/' + name, tf.reduce_max(var))
      tf.scalar_summary('min/' + name, tf.reduce_min(var))
      tf.histogram_summary(name, var)

  def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.

    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
      # This Variable will hold the state of the weights for the layer
      with tf.name_scope('weights'):
        weights = weight_variable([input_dim, output_dim])
        variable_summaries(weights, layer_name + '/weights')
      with tf.name_scope('biases'):
        biases = bias_variable([output_dim])
        variable_summaries(biases, layer_name + '/biases')
      with tf.name_scope('Wx_plus_b'):
        preactivate = tf.matmul(input_tensor, weights) + biases
        tf.histogram_summary(layer_name + '/pre_activations', preactivate)
      activations = act(preactivate, name='activation')
      tf.histogram_summary(layer_name + '/activations', activations)
      return activations

  hidden1 = nn_layer(x, 784, 500, 'layer1')

  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.scalar_summary('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

  # Do not apply softmax activation yet, see below.
  y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

  with tf.name_scope('cross_entropy'):
    # The raw formulation of cross-entropy,
    #
    # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
    #                               reduction_indices=[1]))
    #
    # can be numerically unstable.
    #
    # So here we use tf.nn.softmax_cross_entropy_with_logits on the
    # raw outputs of the nn_layer above, and then average across
    # the batch.
    diff = tf.nn.softmax_cross_entropy_with_logits(y, y_)
    with tf.name_scope('total'):
      cross_entropy = tf.reduce_mean(diff)
    tf.scalar_summary('cross entropy', cross_entropy)

  with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(
        cross_entropy)

  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.scalar_summary('accuracy', accuracy)

  # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
  merged = tf.merge_all_summaries()
  train_writer = tf.train.SummaryWriter(summaries_dir + '/train',
                                        sess.graph)
  test_writer = tf.train.SummaryWriter(summaries_dir + '/test')
  tf.initialize_all_variables().run()

  # Train the model, and also write summaries.
  # Every 10th step, measure test-set accuracy, and write test summaries
  # All other steps, run train_step on training data, & add training summaries

  def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if train or fake_data:
      xs, ys = mnist.train.next_batch(100, fake_data=fake_data)
      k = dropout
    else:
      xs, ys = mnist.test.images, mnist.test.labels
      k = 1.0
    return {x: xs, y_: ys, keep_prob: k}

  for i in range(max_steps):
    if i % 10 == 0:  # Record summaries and test-set accuracy
      summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
      test_writer.add_summary(summary, i)
      print('Accuracy at step %s: %s' % (i, acc))
    else:  # Record train set summaries, and train
      if i % 100 == 99:  # Record execution stats
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        summary, _ = sess.run([merged, train_step],
                              feed_dict=feed_dict(True),
                              options=run_options,
                              run_metadata=run_metadata)
        train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
        train_writer.add_summary(summary, i)
        print('Adding run metadata for', i)
      else:  # Record a summary
        summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
        train_writer.add_summary(summary, i)
  train_writer.close()
  test_writer.close()

In [3]:
if tf.gfile.Exists(summaries_dir):
  tf.gfile.DeleteRecursively(summaries_dir)
tf.gfile.MakeDirs(summaries_dir)
train()

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy at step 0: 0.1044
Accuracy at step 10: 0.7174
Accuracy at step 20: 0.8172
Accuracy at step 30: 0.851
Accuracy at step 40: 0.8776
Accuracy at step 50: 0.8837
Accuracy at step 60: 0.8902
Accuracy at step 70: 0.8947
Accuracy at step 80: 0.8816
Accuracy at step 90: 0.8882
('Adding run metadata for', 99)
Accuracy at step 100: 0.9008
Accuracy at step 110: 0.9115
Accuracy at step 120: 0.9167
Accuracy at step 130: 0.9123
Accuracy at step 140: 0.921
Accuracy at step 150: 0.9162
Accuracy at step 160: 0.9263
Accuracy at step 170: 0.9228
Accuracy at 

tensorboardを起動する。

In [5]:
!tensorboard --logdir=/tmp/mnist_logs

I tensorflow/stream_executor/dso_loader.cc:111] successfully opened CUDA library libcublas.so locally
I tensorflow/stream_executor/dso_loader.cc:111] successfully opened CUDA library libcudnn.so locally
I tensorflow/stream_executor/dso_loader.cc:111] successfully opened CUDA library libcufft.so locally
I tensorflow/stream_executor/dso_loader.cc:111] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:111] successfully opened CUDA library libcurand.so locally
Starting TensorBoard 29 on port 6006
(You can navigate to http://172.17.0.5:6006)
10.83.169.74 - - [20/Oct/2016 10:09:16] "GET / HTTP/1.1" 200 -
10.83.169.74 - - [20/Oct/2016 10:09:16] "GET /webcomponentsjs/webcomponents-lite.min.js HTTP/1.1" 200 -
10.83.169.74 - - [20/Oct/2016 10:09:16] "GET /lib/css/global.css HTTP/1.1" 200 -
10.83.169.74 - - [20/Oct/2016 10:09:16] code 404, message Not Found
10.83.169.74 - - [20/Oct/2016 10:09:16] "GET /dist/bazel-html-imports.html HTTP/1.1" 404 -
10.